# Submission
Lời giải được đề xuất bởi nhóm AIBeginner
- Bùi Cao Doanh (19521366)
- Trịnh Thị Thanh Trúc (19521059)



# Cài thư viện

In [ ]:
# clone repo của chúng tôi về
!git clone https://github.com/caodoanh2001/AITempoRun
%cd AITempoRun

In [ ]:
# tải model
!mkdir models
%cd models
!gdown --id 16prmMkZx9mTOygnRRKjM3Af_nUapcItc
%cd .. 

In [ ]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html
!pip install dict_trie
!python setup.py install
!python setup.py build develop

# Traning
Chúng tôi nhận thấy dữ liệu BTC bị gán nhãn sai rất nhiều. Do đó chúng tôi tiến hành thực hiện NMS để loại bỏ nhiễu đi, với ngưỡng là 0.1. Bên cạnh đó áp dụng 1 giải thuật nho nhỏ để xếp lại các tọa độ theo đúng chiều kim đồng hồ

## Bước chúng tôi tiền xử lý: NMS + xoay cho đúng chiều kim đồng hồ

In [ ]:
import glob
import os
import json
from tqdm import tqdm_notebook
from functools import reduce
import operator
import math
import cv2
import numpy as np
import nms

dir_imgs = '/content/drive/MyDrive/AITempoRun/train/images/'
dir_labels = '/content/drive/MyDrive/AITempoRun/train/labels/'
list_images = glob.glob('/content/drive/MyDrive/AITempoRun/train/images/*')
list_labels = glob.glob('/content/drive/MyDrive/AITempoRun/train/labels/*')

total_convert = []
new_data = []

for img_path in tqdm_notebook(list_images):
    img = cv2.imread(img_path)
    list_labels = []
    file_name = os.path.splitext(img_path.split('/')[-1])[0]
    label_file = file_name + '.txt'
    img_name = file_name + '.jpg'
    with open(os.path.join(dir_labels, label_file), 'r') as f:
        labels = f.read()
    labels = [label.split(',') for label in labels.split('\n')]
    img_points = []
    for label in labels:
        points = [list(map(int,label[:-1][i:i + 2])) for i in range(0, len(label[:-1]), 2)]
        center = tuple(map(operator.truediv, reduce(lambda x, y: map(operator.add, x, y), points), [len(points)] * 2))
        points = sorted(points, key=lambda point: (-135 - math.degrees(math.atan2(*tuple(map(operator.sub, point, center))[::-1]))) % 360)
        points = [item for sublist in points for item in sublist] + [1.]
        img_points.append(points)

    nms_points = nms.nms(img_points, 0.1)
    img_points = [img_points[i] for i in [i for i, x in enumerate(nms_points) if x]]

Chúng tôi lưu vô file label (txt) và convert sang COCO để train

In [178]:
labels = open('labels.txt', 'r').read().split('\n')

In [ ]:
import cv2
from tqdm import tqdm_notebook
dict_coco = {'images': [], 'annotations': [], 'categories': [{'id': 1,
'keypoints': ['mean',
'xmin',
'x2',
'x3',
'xmax',
'ymin',
'y2',
'y3',
'ymax',
'cross'],
'name': 'text',
'supercategory': 'beverage'}]}

img_dir = '/content/drive/MyDrive/AITempoRun/train/images'
img_id = {}
for i, img in enumerate(glob.glob(os.path.join(img_dir, '*'))):
    img_id[img.split('/')[-1]] = i

for i, label in tqdm_notebook(enumerate(labels)):
    label = label.split(',')
    filename = label[0]
    img = cv2.imread(os.path.join(img_dir, filename))
    width, height = img.shape[1], img.shape[0]
    segmentation = list(map(int, label[1:]))
    dict_coco['images'].append({
        'file_name': filename,
        'height': height,
        'id': img_id[filename],
        'width': 3196
    })
    dict_coco['annotations'].append(
    {
        'area': 0,
        'bbox': [segmentation[0],segmentation[1],segmentation[4],segmentation[5]],
        'bezier_pts': [segmentation[0],segmentation[1],\
                       int((segmentation[0]+segmentation[2])/2), int((segmentation[1]+segmentation[3])/2),\
                       segmentation[2], segmentation[3], \
                       int((segmentation[4]+segmentation[6])/2), int((segmentation[5]+segmentation[7])/2),\
                       segmentation[6], segmentation[7]],
        'category_id': 1,
        'id': i,
        'image_id': img_id[filename],
        'iscrowd': 0,
        'segmentation': segmentation
    })
    
with open('train.json', 'w') as f:
    json.dump(dict_coco,f )

Để train thì thực thi câu lệnh sau. Sử dụng pretrain từ ABCNet

In [ ]:
!python tools/train_net.py --config-file cfgs/config.yaml MODEL.WEIGHTS weights/pretrain_attn_R_50.pth

# Testing

## Import thư viện và các biến, function phụ trợ

In [3]:
from detectron2.engine.defaults import DefaultPredictor
from detectron2.data.detection_utils import read_image
from adet.config import get_cfg
from tqdm import tqdm_notebook
import time
import glob
import os

In [4]:
def prepare_cfg(cfg_file, model_path):
    cfg = get_cfg()
    cfg.merge_from_file(cfg_file)
    cfg.merge_from_list(['MODEL.WEIGHTS', model_path])
    cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.FCOS.INFERENCE_TH_TEST = 0.5
    cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = 0.5
    cfg.freeze()
    return cfg

In [5]:
cfg_file = './cfgs/config.yaml'
model_path = './models/finalModel.pth'
path_test = '/content/drive/MyDrive/AITempoRun/public_test' # đường dẫn thư mục ảnh test 

## Thực thi ra file submission.txt

In [7]:
cfg = prepare_cfg(cfg_file, model_path)
predictor = DefaultPredictor(cfg)

print('Read predictor successfully. We are ready for predictions')

list_txts = []
start_time = time.time()
for img_path in tqdm_notebook(glob.glob(os.path.join(path_test,'*'))):
    img = read_image(img_path, format="BGR")
    try:
        predictions = predictor(img)
        raw_points = predictions['instances'].beziers.cpu().detach().numpy()
        for p in raw_points:
            p = [list(map(int,p[i:i + 2])) for i in range(0, len(p), 2)]
            points = [p[0]] + [p[3]] + [p[4]] + [p[7]]
            export_str = img_path.split('/')[-1] + ','
            for _, _p in enumerate(points):
                export_str += ','.join(list(map(str,list(map(int,_p)))))
                if not (_ == len(points) - 1):
                    export_str += ','
            list_txts.append(export_str)
    except:
        print('Error at', img_path)
end_time = time.time()

print('Done. We are writing our predictions to submission.txt')

list_txts = sorted(list_txts)
f_submission = open('submission.txt', 'w')
for _, line in enumerate(list_txts):
    f_submission.write(line + '\n') if _ != len(list_txts) -1 else f_submission.write(line)
f_submission.close()

print('Done! Thanks for waiting, the predict time is {}'.format(end_time - start_time))

Config './cfgs/config.yaml' has no VERSION. Assuming it to be compatible with latest v2.
The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


Read predictor successfully. We are ready for predictions


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Error at /content/drive/MyDrive/AITempoRun/public_test/test_0121.jpg


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "



Done. We are writing our predictions to submission.txt


NameError: ignored